In [10]:
#----------Data----------#

#Chemical elements
element = [:carbon, :copper, :manganese]

#Minimum grade of chemical elements required
minGrade = Dict(:carbon => 0.02, :copper => 0.004, :manganese => 0.012)

#Maximum grade of chemical elements required
maxGrade = Dict(:carbon => 0.03, :copper => 0.006, :manganese => 0.0165)

#Raw materials
mat = [:ironAlloy1, :ironAlloy2, :ironAlloy3, :copper1, :copper2, :aluminium1, :aluminium2]

#Carbon grade of raw materials
carbGrade = Dict(:ironAlloy1 => 0.025, :ironAlloy2 => 0.03, :ironAlloy3 => 0, 
          :copper1 => 0, :copper2 => 0, :aluminium1 => 0, :aluminium2 => 0)

#Copper grade of raw materials
copGrade = Dict(:ironAlloy1 => 0, :ironAlloy2 => 0, :ironAlloy3 => 0.003, 
          :copper1 => 0.9, :copper2 => 0.96, :aluminium1 => 0.004, :aluminium2 => 0.006)

#Manganese grade of raw materials
mangGrade = Dict(:ironAlloy1 => 0.013, :ironAlloy2 => 0.008, :ironAlloy3 => 0, 
          :copper1 => 0, :copper2 => 0.04, :aluminium1 => 0.012, :aluminium2 => 0)

#Amount of raw materials available in tons
matQuant = Dict(:ironAlloy1 => 400, :ironAlloy2 => 300, :ironAlloy3 => 600, 
          :copper1 => 500, :copper2 => 200, :aluminium1 => 300, :aluminium2 => 250)

#Cost per ton of raw materials in dollars
matCost = Dict(:ironAlloy1 => 200, :ironAlloy2 => 250, :ironAlloy3 => 150, 
          :copper1 => 220, :copper2 => 240, :aluminium1 => 200, :aluminium2 => 165)

#Amount of steel ordered in tons
orderedQuant = 500



#----------Model----------#

using JuMP
m = Model()

#Variables
@defVar(m, matUsed[mat] >= 0)                                           #Amount of a material used in tons 

#Expressions
@defExpr(prodQuant, sum{matUsed[i], i in mat})                               #Total material amount used in production
@defExpr(totalCost, sum{matUsed[i]*matCost[i], i in mat})                    #Total cost
@defExpr(prodCarbGrade, sum{matUsed[i]*carbGrade[i], i in mat} / orderedQuant)   #Carbon grade of the steel produced
@defExpr(prodCopGrade, sum{matUsed[i]*copGrade[i], i in mat} / orderedQuant)     #Copper grade of the steel produced
@defExpr(prodMangGrade, sum{matUsed[i]*mangGrade[i], i in mat} / orderedQuant)   #Manganese grade of the steel produced

#Constraints:

#Fulfil demand
@addConstraint(m, prodQuant == orderedQuant)

#Constraints on minimum and maximum grade of elements
 #a)carbon
@addConstraint(m, minGrade[:carbon] <= prodCarbGrade <= maxGrade[:carbon])
 #b)copper
@addConstraint(m, minGrade[:copper] <= prodCopGrade <= maxGrade[:copper])
 #c) manganese
@addConstraint(m, minGrade[:manganese] <= prodMangGrade <= maxGrade[:manganese])

#Constraint on amount of each material used
@addConstraint(m, quantity[i in 1:7], matUsed[mat[i]] <= matQuant[mat[i]])


#Cost function to minimize
@setObjective(m, Min, totalCost)


#----------Display result----------#
solve(m)
println("Materials used: ")    
println(getValue(matUsed))
println("Total cost: ", getValue(totalCost), " dollars" )

Materials used: 
matUsed: 1 dimensions, 7 entries:
 [aluminium1] = 57.462425725270904
 [aluminium2] = 0.0
 [   copper1] = 0.0
 [   copper2] = 2.761272282418734
 [ironAlloy1] = 400.0
 [ironAlloy2] = 0.0
 [ironAlloy3] = 39.77630199231035
Total cost: 98121.63579168123 dollars
